### SageMaker fine tune ChatGLM

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备requirements.txt
3. 准备s5cmd utility
4. 下载HF model，并上传到S3(optional)

In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

  Created wheel for sagemaker: filename=sagemaker-2.153.0-py2.py3-none-any.whl size=1008105 sha256=b5c59266f9d3f8b737aad3656323127b1d96ebc365d2213038e25c3985ac00cf
  Stored in directory: /home/ec2-user/.cache/pip/wheels/6a/0b/81/d7a0f0e734bc9f5ea74716acef6b397a6d473cb8e6b9b3f121
Successfully built sagemaker
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.0
    Uninstalling cloudpickle-2.2.0:
      Successfully uninstalled cloudpickle-2.2.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.146.0
    Uninstalling sagemaker-2.146.0:
      Successfully uninstalled sagemaker-2.146.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.11.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.2 which is incompatible.


In [3]:
#print('s3://{}/llm/models/'.format(sagemaker_session.default_bucket()))
#!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/
!curl -L https://github.com/peak/s5cmd/releases/download/v2.0.0/s5cmd_2.0.0_Linux-64bit.tar.gz | tar -xz && mv s5cmd ./ptuning/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 4176k  100 4176k    0     0  5928k      0 --:--:-- --:--:-- --:--:-- 11.9M


In [4]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123
sagemaker-us-west-2-687912291502


In [5]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.1 MB/s eta 0:00:00


In [6]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_cache_path = Path("./model")
local_cache_path.mkdir(exist_ok=True)

model_name = "THUDM/chatglm-6b"#

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.model"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    #allow_patterns=allow_patterns,
)

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

In [7]:
print(model_download_path)
model_uri="s3://"+bucket+"/llm/model/chatglm/orignal/"
print(model_uri)

model/models--THUDM--chatglm-6b/snapshots/4d0fc39a58dcb747ab74ab2c4587bd66dcfa7e74
s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/


In [ ]:
#using s5cmd util to fast upload model to s3
! ./ptuning/s5cmd sync ./model/models--THUDM--chatglm-6b/snapshots/4d0fc39a58dcb747ab74ab2c4587bd66dcfa7e74/ s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/
#!rm -rf model

### chatglm 官方P-tuning v2方式（单机单卡）
1:安装依赖lib   
2:准备数据集(本例以ADGEN 文本生成数据集为例，将解压后的 AdvertiseGen 目录放到本目录  
3:修改并bash运行 train.sh  

In [24]:
!cd ptuning/ && wget "https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1"
!cd ptuning/ && mv "index.html?dl=1" dataset.tar.gz
!cd ptuning/ && tar -xvf dataset.tar.gz
!./ptuning/s5cmd sync ./ptuning/AdvertiseGen/ s3://{bucket}/llm/chatglm/datasets/ 
!rm -rf cd ./ptuning/dataset.tar.gz


--2023-05-19 01:20:02--  https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1
Resolving cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)... 166.111.6.101, 2402:f000:1:406:166:111:6:101
Connecting to cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)|166.111.6.101|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cloud.tsinghua.edu.cn/seafhttp/files/12408e4c-4a4a-4a3e-9c7d-4b03f57ae207/AdvertiseGen.tar.gz [following]
--2023-05-19 01:20:03--  https://cloud.tsinghua.edu.cn/seafhttp/files/12408e4c-4a4a-4a3e-9c7d-4b03f57ae207/AdvertiseGen.tar.gz
Reusing existing connection to cloud.tsinghua.edu.cn:443.
HTTP request sent, awaiting response... 200 OK
Length: 17069994 (16M) [application/octet-stream]
Saving to: ‘index.html?dl=1’

100%[======================================>] 17,069,994  5.53MB/s   in 2.9s   

2023-05-19 01:20:07 (5.53 MB/s) - ‘index.html?dl=1’ saved [17069994/17069994]

AdvertiseGen/
AdvertiseGen/train.json
AdvertiseGen/dev.json
Name:
  sync 

In [19]:
# define Training Job Name 
import time
from sagemaker.huggingface import HuggingFace
instance_type="ml.g4dn.2xlarge"

job_name = f'huggingface-chatglm-simple-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
#define the model s3 path which will store your trained model asset
#Note: you should use your real s3 path to configure model_s3_path
model_s3_path='s3://{}/llm/models/chatglm/simple/'.format(sagemaker_session.default_bucket())
output_dir = '/opt/ml/model/adgen-chatglm-6b-ft'
model_name_or_path = 'THUDM/chatglm-6b'


instance_count = 1
#define the enviroment variables for your scripts.
environment = {
              'MODEL_S3_PATH'          : model_uri,
              'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
              #'LD_LIBRARY_PATH'        : '${LD_LIBRARY_PATH}:/opt/conda/lib/',
              'TRAIN_DATASET'          : '/opt/ml/input/data/AdvertiseGen/train.json',
              'TEST_DATASET'           : '/opt/ml/input/data/AdvertiseGen/dev.json',
              'PROMPT_COLUMN'          : 'content',
              'RESPONSE_COLUMN'        : 'summary',
              'MODEL_NAME_OR_PATH'     : model_name_or_path,
              'OUTPUT_DIR'             : output_dir,
              'MODEL_OUTPUT_S3_PATH'   : model_s3_path,
              'TRAIN_STEPS'            : '50'
}

inputs={
   'AdvertiseGen': f"s3://{bucket}/llm/chatglm/datasets/"
}


# create the Estimator
from sagemaker.pytorch import PyTorch

huggingface_estimator = HuggingFace(
    entry_point          = 'start_simple.py',          # user endpoint script
    source_dir           = './ptuning',               # directory which includes all the files needed for training
    instance_type        = instance_type, # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,           # Iam role used in training job to access AWS ressources, e.g. S3
    script_mode          = True,
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    environment = environment
)

huggingface_estimator.fit(inputs=inputs)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker:Creating training-job with name: huggingface-chatglm-simple-2023-05-19-0-2023-05-19-00-55-21-706


2023-05-19 00:55:24 Starting - Starting the training job...
2023-05-19 00:55:39 Starting - Preparing the instances for training......
2023-05-19 00:56:54 Downloading - Downloading input data
2023-05-19 00:56:54 Training - Downloading the training image.....................
2023-05-19 01:00:06 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-19 01:00:29,809 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-19 01:00:29,827 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-19 01:00:29,838 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-19 01:00:29,840 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-19 01:00:30,378 sagemaker-training-toolkit INFO     Installing dependencies

In [20]:
huggingface_estimator.model_data

's3://sagemaker-us-west-2-687912291502/huggingface-chatglm-simple-2023-05-19-0-2023-05-19-00-55-21-706/output/model.tar.gz'

In [22]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/chatglm/simple/adgen-chatglm-6b-ft/

                           PRE checkpoint-100/
                           PRE checkpoint-200/
                           PRE checkpoint-300/
                           PRE checkpoint-50/
2023-05-19 01:09:31        188 all_results.json
2023-05-19 01:09:31        869 config.json
2023-05-19 01:09:31       4276 configuration_chatglm.py
2023-05-19 01:09:31        142 generation_config.json
2023-05-19 01:09:31    2706249 ice_text.model
2023-05-19 01:09:31      57620 modeling_chatglm.py
2023-05-19 01:09:31  117441341 pytorch_model.bin
2023-05-19 01:09:31      15054 quantization.py
2023-05-19 01:09:31        125 special_tokens_map.json
2023-05-19 01:09:31      17047 tokenization_chatglm.py
2023-05-19 01:09:31        495 tokenizer_config.json
2023-05-19 01:09:31        188 train_results.json
2023-05-19 01:09:31       1155 trainer_state.json
2023-05-19 01:09:31       3771 training_args.bin


### chatglm 官方deepspeed方式（全参数的Finetune,单机多卡）
1: 准备deepspeed lib，并修改deepspeed.json    
2：数据集（以上一致）  
3：entrypoint start-single-node.py,设置num-gpus  
4：触发bash ds_train_finetune_single_node.sh 

In [13]:
instance_type = 'ml.g5.48xlarge'
if instance_type in [
    "ml.p3.16xlarge",
    "ml.p3dn.24xlarge",
    "ml.g5.48xlarge",
    "ml.p4d.24xlarge"    
]:
    processes_per_host = 8
elif instance_type == "ml.p2.16xlarge":
    processes_per_host = 16
elif instance_type == "ml.g4dn.2xlarge":
    processes_per_host = 1
else:
    processes_per_host = 4
    

print("processes_per_host is set to:", processes_per_host)

processes_per_host is set to: 8


In [14]:
# define Training Job Name 
job_name = f'huggingface-chatglm-deepspeed-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
#define the model s3 path which will store your trained model asset
#Note: you should use your real s3 path to configure model_s3_path
model_s3_path='s3://{}/llm/models/chatglm/deepspeed-single-node/'.format(sagemaker_session.default_bucket())
output_dir = '/tmp/model/adgen-chatglm-6b-ft'
model_name_or_path = 'THUDM/chatglm-6b'
#model_name_or_path = "/tmp/chatglm/"


instance_count = 1
#define the enviroment variables for your scripts.
environment = {
              
              'MODEL_S3_PATH'          : model_uri,
              'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
              #'LD_LIBRARY_PATH'        : '${LD_LIBRARY_PATH}:/opt/conda/lib/',
              'NUM_GPUS'               : str(processes_per_host),
              'TRAIN_DATASET'          : '/opt/ml/input/data/AdvertiseGen/train.json',
              'TEST_DATASET'           : '/opt/ml/input/data/AdvertiseGen/dev.json',
              'PROMPT_COLUMN'          : 'content',
              'RESPONSE_COLUMN'        : 'summary',
              'MODEL_NAME_OR_PATH'     : model_name_or_path,
              'OUTPUT_DIR'             : output_dir,
              'MODEL_OUTPUT_S3_PATH'   : model_s3_path,
              'TRAIN_STEPS'            :'20'
}

inputs={
   'AdvertiseGen': f"s3://{bucket}/llm/chatglm/datasets/"
}


# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'start-single-node.py',          # user endpoint script
    source_dir           = './ptuning',               # directory which includes all the files needed for training
    instance_type        = instance_type, # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    script_mode          = True,
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    environment = environment,
)
huggingface_estimator.fit(inputs=inputs)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-chatglm-deepspeed-2023-05-1-2023-05-11-09-58-06-102


Using provided s3_resource
2023-05-11 09:58:08 Starting - Starting the training job......
2023-05-11 09:58:44 Starting - Preparing the instances for training......
2023-05-11 10:00:02 Downloading - Downloading input data...
2023-05-11 10:00:22 Training - Downloading the training image..................
2023-05-11 10:03:18 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-11 10:04:05,588 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-11 10:04:05,651 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-11 10:04:05,661 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-11 10:04:05,664 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-11 10:04:06,206 sagemaker-training-toolkit 

In [17]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/chatglm/deepspeed-single-node/adgen-chatglm-6b-ft/

                           PRE checkpoint-20/
2023-05-11 10:20:54        192 all_results.json
2023-05-11 10:20:54        870 config.json
2023-05-11 10:20:54       4276 configuration_chatglm.py
2023-05-11 10:20:54        142 generation_config.json
2023-05-11 10:20:54    2706249 ice_text.model
2023-05-11 10:20:54      57620 modeling_chatglm.py
2023-05-11 10:20:54 12346621179 pytorch_model-00001-of-00002.bin
2023-05-11 10:20:54 12346585635 pytorch_model-00002-of-00002.bin
2023-05-11 10:20:54      33416 pytorch_model.bin.index.json
2023-05-11 10:20:54      15054 quantization.py
2023-05-11 10:20:54        125 special_tokens_map.json
2023-05-11 10:20:54      17047 tokenization_chatglm.py
2023-05-11 10:20:54        495 tokenizer_config.json
2023-05-11 10:20:54        192 train_results.json
2023-05-11 10:20:54        808 trainer_state.json
2023-05-11 10:20:54       4795 training_args.bin


### chatglm deepspeed 多机多卡改造
1: 准备deepspeed lib，并修改deepspeed.json    
2：数据集（以上一致）  
3：entrypoint start.py,设置torch distribute launch configure  
4：触发bash ds_train_finetune.sh 

In [15]:
# define Training Job Name 
job_name = f'huggingface-chatglm-deepspeed-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
#define the model s3 path which will store your trained model asset
#Note: you should use your real s3 path to configure model_s3_path
model_s3_path='s3://{}/llm/models/chatglm/deepspeed-mutip-nodes/'.format(sagemaker_session.default_bucket())
output_dir = '/tmp/model/adgen-chatglm-6b-ft'
model_name_or_path = 'THUDM/chatglm-6b'


instance_count = 2
#define the enviroment variables for your scripts.
environment = {
              'NODE_NUMBER'            : str(instance_count),
              'MODEL_S3_PATH'          : model_uri,
              'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
              #'LD_LIBRARY_PATH'        : '${LD_LIBRARY_PATH}:/opt/conda/lib/',
              'NUM_GPUS'               : str(processes_per_host),
              'TRAIN_DATASET'          : '/opt/ml/input/data/AdvertiseGen/train.json',
              'TEST_DATASET'           : '/opt/ml/input/data/AdvertiseGen/dev.json',
              'PROMPT_COLUMN'          : 'content',
              'RESPONSE_COLUMN'        : 'summary',
              'MODEL_NAME_OR_PATH'     : model_name_or_path,
              'OUTPUT_DIR'             : output_dir,
              'MODEL_OUTPUT_S3_PATH'   : model_s3_path,
              'TRAIN_STEPS'            :'50'
}

inputs={
   'AdvertiseGen': f"s3://{bucket}/llm/chatglm/datasets/"
}


# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'start.py',          # user endpoint script
    source_dir           = './ptuning',               # directory which includes all the files needed for training
    instance_type        = instance_type, # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    script_mode          = True,
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    environment = environment,
)
huggingface_estimator.fit(inputs=inputs)

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-chatglm-deepspeed-2023-05-1-2023-05-11-10-22-23-381


2023-05-11 10:22:25 Starting - Starting the training job......
2023-05-11 10:23:04 Starting - Preparing the instances for training......
2023-05-11 10:24:19 Downloading - Downloading input data...
2023-05-11 10:24:33 Training - Downloading the training image..................
2023-05-11 10:27:50 Training - Training image download completed. Training in progress.......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-11 10:28:41,303 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-11 10:28:41,364 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-11 10:28:41,375 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-11 10:28:41,377 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-11 10:28:41,818 sagemaker-training-toolkit INFO     Installing depen

In [16]:
#!aws s3 rm s3://sagemaker-us-west-2-687912291502/llm/models/chatglm/deepspeed/adgen-chatglm-6b-ft/checkpoint-50 --recursive
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/chatglm/deepspeed-mutip-nodes/adgen-chatglm-6b-ft/

                           PRE checkpoint-50/
2023-05-11 10:56:40        194 all_results.json
2023-05-11 10:56:40        870 config.json
2023-05-11 10:56:40       4276 configuration_chatglm.py
2023-05-11 10:56:40        142 generation_config.json
2023-05-11 10:56:40    2706249 ice_text.model
2023-05-11 10:56:40      57620 modeling_chatglm.py
2023-05-11 10:56:40 12346621179 pytorch_model-00001-of-00002.bin
2023-05-11 10:56:40 12346585635 pytorch_model-00002-of-00002.bin
2023-05-11 10:56:40      33416 pytorch_model.bin.index.json
2023-05-11 10:56:40      15054 quantization.py
2023-05-11 10:56:40        125 special_tokens_map.json
2023-05-11 10:56:40      17047 tokenization_chatglm.py
2023-05-11 10:56:40        495 tokenizer_config.json
2023-05-11 10:56:40        194 train_results.json
2023-05-11 10:56:40       1123 trainer_state.json
2023-05-11 10:56:40       4795 training_args.bin


In [ ]:
##for local test only
!cd ./ChatGLM-6B/ptuning/ && bash ./ds_train_finetune.sh

In [153]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/

                           PRE chatglm/
                           PRE model/
                           PRE models/
